# Д.З. №7. Анализ отзывов на лекарства

## Данные

Ссылка на датасет: [Kaggle: Drug Reviews](https://www.kaggle.com/datasets/jessicali9530/kuc-hackathon-winter-2018)

## Задание

Реализовать модель для прогноза колонки `rating` на основе текста из колонки `review`. Рассматриваем это как задачу **регрессии**.

Необходимо применить **глубокую NLP-модель**. Подход можно выбрать самостоятельно:  
- RNN (например, LSTM, GRU)  
- трансформеры (например, BERT, RoBERTa и т.д.)

---

## Дополнительные (опциональные) задачи

Можно выполнить одну или несколько дополнительных задач. Это не обязательно, но может значительно повысить итоговый балл.

### 1. Тематическое моделирование (Topic Modeling)

Попробовать тематическую кластеризацию текстов из `review`:

- Сначала получить **эмбеддинги текста** (подходящие примеры — в конце страницы с заданием).
- Использовать **англоязычные модели**, так как тексты на английском.
- Провести кластеризацию эмбеддингов. Возможные методы:
  - C-TF-IDF
  - LDA (можно использовать реализацию с трансформерными эмбеддингами)

**Как оценить качество кластеров:**  
Посмотрите на ключевые слова (или фразы) внутри каждого кластера. Если они логично и последовательно отражают общую тему отзывов — кластеризация считается удачной.

### 2. Дистилляция трансформера в LSTM

Можно попробовать **дистиллировать большую трансформерную модель** в более лёгкую LSTM-архитектуру:

- Либо использовать заготовленный код (ссылка в конце задания)
- Либо найти/адаптировать готовое решение

Это может быть полезно для ускорения вывода модели или снижения требований к ресурсам.


## Загрузка и первичная проверка датасета

На этом шаге мы скачиваем датасет с Kaggle и загружаем train и test CSV файлы.
Данные читаются целиком без фильтрации строк и колонок.
Мы проверяем размеры выборок, базовую структуру и визуально осматриваем первые записи.
Никакая очистка или предобработка данных здесь не выполняется.


In [1]:
from pathlib import Path
from IPython.display import Markdown, display

from scripts.dataset import download_dataset, load_raw_splits


def md(text: str) -> None:
    display(Markdown(text))


project_root = Path.cwd().resolve()
raw_data_dir = project_root / "data" / "raw"

download_dataset(
    dataset_id="jessicali9530/kuc-hackathon-winter-2018",
    target_dir=raw_data_dir,
    force=False,
)

train_csv = raw_data_dir / "drugsComTrain_raw.csv"
test_csv = raw_data_dir / "drugsComTest_raw.csv"

train_df, test_df = load_raw_splits(
    train_path=train_csv,
    test_path=test_csv,
)

md("### Train dataset overview")
md(f"Shape: {train_df.shape}")
display(train_df.head())

md("### Test dataset overview")
md(f"Shape: {test_df.shape}")
display(test_df.head())


Dataset already exists. Skipping download.

Loading training dataset

Loading dataset from `/home/garret/git/mfti/llm_hw7_medicine_reviews/data/raw/drugsComTrain_raw.csv`

Loaded dataframe with 161297 rows and 7 columns

Ratings converted to numeric. Missing values after conversion: 0

...

Shape: (161297, 7)

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


### Test dataset overview

Shape: (53766, 7)

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10,28-Feb-12,22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8,17-May-09,17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9,29-Sep-17,3
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9,5-Mar-17,35
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9,22-Oct-15,4
